In [1]:
# SIMULATION PARAMETERS
verbosity         = 2     # Set debug print statement verbosity level (0 = Standard, -1 = Off)
use_mass_units    = True  # Toggle whether calculations / results are given in units of pi-axion mass (True) or eV (False)
use_natural_units = True  # Toggle whether calculations / results are given in c = h = G = 1 (True) or SI units (False)   || NOTE: full SI/phsyical unit support is still WIP!!

In [2]:
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from scipy.signal import spectrogram
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import matplotlib.image as image
from piaxi_numerics import set_params, solve_system, get_text_params
import os

debug_level = verbosity

In [3]:
# Set constants of model
e    = 0.3      # dimensionless electron charge
F    = 1e17     # pi-axion decay constant (GeV) >= 10^11
#p_t  = 0.4     # total local DM density (GeV/cm^3)
p_t = 1e11
p_unit = 1.906e-12 if use_natural_units else 1. # convert densities from units of [1/cm^3] to [(eV/hc)^3]
## --> TODO: Could/Should we support spatially dependent distributions?

## Tuneable constants
# millicharge, vary to enable/disable charged species (10e-15 = OFF, 10e-25 = ON)
#eps  = 1e-25   # (unitless)
eps  = 1

# Coupling constants
#L3   = 1e11    # (GeV)
L3   = 1e11
#L4   = 1e11    # (GeV)
L4   = 1e11
l1   = 1       #
l2   = 1       #
l3   = 1       #
l4   = 1       #

# Unit scaling:
GeV  = 1e9     # GeV -> eV
#GeV = 1
F   *= GeV
p_t *= GeV
p_t *= p_unit  # 1/cm^3 -> (eV/hc)^3
L3  *= GeV
L4  *= GeV

# Fundamental constants
c_raw = c = np.float64(2.998e10)    # Speed of light in a vacuum [cm/s]
h_raw = h = np.float64(4.136e-15)   # Planck's constant [eV/Hz]
G_raw = G = np.float64(1.0693e-19)  # Newtonian constant [cm^5 /(eV s^4)]
manual_set = False
if manual_set: # Manually toggle units
    unitful_c = False
    unitful_h = False
    unitful_G = False
if use_natural_units:
    unitful_c = unitful_h = unitful_G = False
else:
    unitful_c = unitful_h = unitful_G = True
    
c_u = c if unitful_c else 1.
h_u = h if unitful_h else 1.
G_u = G if unitful_G else 1.

In [4]:
## Dark SM Parameters
sample_qmass = False # TODO
sample_qcons = False
# Mass scaling parameters
m_scale = 1e-40             # dark quark mass scale (eV) <= 10-20

# SM quark masses for all 3 generations
qm = m_scale*np.array([1., 2., 40.]) if not sample_qmass else m_scale*np.array([0., 0., 0.]) # TODO

# dSM quark scaling constants (up, down, strange, charm, bottom, top) sampled from uniform distribution [0.7, 1.3]
qc = np.array([1., 1., 1., 0., 0., 0.]) if not sample_qcons else np.random.uniform(0.7, 1.3, (6,))
#qc = np.array([1., 1., 1., 1., 1., 1.]) if not sample_qcons else np.random.uniform(0.7, 1.3, (6,))

# Dark quark masses (up, down, strange, charm, bottom, top)
dqm = np.array([qm[0]*qc[0], qm[0]*qc[1], qm[1]*qc[2], qm[1]*qc[3], qm[2]*qc[4], qm[2]*qc[5]])

# Scaling parameters
xi     = np.array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])  # Charged species scaling paramters
eps_c  = np.array([+1., +1., +1., -1., -1., -1., -1., +1., -1.])  # Millicharge sign

In [5]:
## Pi-axion Species Mass Definitions
m_r = np.array([0., 0., 0., 0., 0.], dtype=float)                   # real neutral
m_n = np.array([0., 0., 0., 0., 0., 0.], dtype=float)               # complex neutral
m_c = np.array([0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float)   # charged
# Pi-axion Species Labels
s_l = np.array([np.full_like(m_r, '', dtype=str), np.full_like(m_n, '', dtype=str), np.full_like(m_c, '', dtype=str)], dtype=object)

## Real Neutral Masses
# pi_3
m_r[0]    = np.sqrt(((qc[0] + qc[1])*qm[0])*F) if 0. not in [qc[0], qc[1]] else 0.
s_l[0][0] = '$\pi_{3}$'
# pi_8
m_r[1]    = np.sqrt(((qc[0] + qc[1])*qm[0] + qc[2]*qm[1])*F) if 0. not in [qc[0], qc[1], qc[2]] else 0.
s_l[0][1] = '$\pi_{8}$'
# pi_29
m_r[2]    = np.sqrt(((qc[3]*qm[1]) + (qc[4]*qm[2]))*F) if 0. not in [qc[3], qc[4]] else 0.
s_l[0][2] = '$\pi_{29}$'
# pi_34
m_r[3]    = np.sqrt(((qc[3]*qm[1]) + ((qc[4] + qc[5])*qm[2]))*F) if 0. not in [qc[3], qc[4], qc[5]] else 0.
s_l[0][3] = '$\pi_{34}$'
# pi_35
m_r[4]    = np.sqrt(((qc[0]+qc[1])*qm[0] + (qc[2]+qc[3])*qm[1] + (qc[4]+qc[5])*qm[2])*F) if 0. not in [qc[0], qc[1], qc[2], qc[3], qc[4], qc[5]] else 0.
s_l[0][4] = '$\pi_{35}$'

## Complex Neutral Masses
# pi_6  +/- i*pi_7
m_n[0]    = np.sqrt((qc[1]*qm[0] + qc[2]*qm[1]) * F) if 0. not in [qc[1], qc[2]] else 0.
s_l[1][0] = '$\pi_6 \pm i\pi_7$'
# pi_9  +/- i*pi_10
m_n[1]    = np.sqrt((qc[0]*qm[0] + qc[3]*qm[1]) * F) if 0. not in [qc[0], qc[3]] else 0.
s_l[1][1] = '$\pi_9 \pm i\pi_{10}$'
# pi_17 +/- i*pi_18
m_n[2]    = np.sqrt((qc[1]*qm[0] + qc[4]*qm[2]) * F) if 0. not in [qc[1], qc[4]] else 0.
s_l[1][2] = '$\pi_{17} \pm i\pi_{18}$'
# pi_19 +/- i*pi_20
m_n[3]    = np.sqrt((qc[2]*qm[1] + qc[4]*qm[2]) * F) if 0. not in [qc[2], qc[4]] else 0.
s_l[1][3] = '$\pi_{19} \pm i\pi_{20}$'
# pi_21 +/- i*pi_22
m_n[4]    = np.sqrt((qc[0]*qm[0] + qc[5]*qm[2]) * F) if 0. not in [qc[0], qc[5]] else 0.
s_l[1][4] = '$\pi_{21} \pm i\pi_{22}$'
# pi_30 +/- i*pi_31
m_n[5]    = np.sqrt((qc[3]*qm[1] + qc[5]*qm[2]) * F) if 0. not in [qc[3], qc[5]] else 0.
s_l[1][5] = '$\pi_{30} \pm i\pi_{31}$'

## Charged Masses
# pi_1  +/- i*pi_2
m_c[0]    = np.sqrt((qc[0]*qm[0] + qc[1]*qm[0])*F + 2*xi[0]*(e*eps*F)**2) if 0. not in [qc[0], qc[1]] and abs(eps) < 0.1 else 0.
s_l[2][0] = '$\pi_1 \pm i\pi_2$'
# pi_4  +/- i*pi_5
m_c[1]    = np.sqrt((qc[0]*qm[0] + qc[2]*qm[1])*F + 2*xi[1]*(e*eps*F)**2) if 0. not in [qc[0], qc[2]] and abs(eps) < 0.1 else 0.
s_l[2][1] = '$\pi_4 \pm i\pi_5$'
# pi_15 +/- i*pi_16
m_c[2]    = np.sqrt((qc[0]*qm[0] + qc[4]*qm[2])*F + 2*xi[2]*(e*eps*F)**2) if 0. not in [qc[0], qc[4]] and abs(eps) < 0.1 else 0.
s_l[2][2] = '$\pi_{15} \pm i\pi_{16}$'
# pi_11 +/- i*pi_12
m_c[3]    = np.sqrt((qc[1]*qm[0] + qc[3]*qm[2])*F + 2*xi[3]*(e*eps*F)**2) if 0. not in [qc[1], qc[3]] and abs(eps) < 0.1 else 0.
s_l[2][3] = '$\pi_{11} \pm i\pi_{12}$'
# pi_23 +/- i*pi_24
m_c[4]    = np.sqrt((qc[1]*qm[0] + qc[5]*qm[2])*F + 2*xi[4]*(e*eps*F)**2) if 0. not in [qc[1], qc[5]] and abs(eps) < 0.1 else 0.
s_l[2][4] = '$\pi_{23} \pm i\pi_{24}$'
# pi_13 +/- i*pi_14
m_c[5]    = np.sqrt((qc[2]*qm[1] + qc[3]*qm[1])*F + 2*xi[5]*(e*eps*F)**2) if 0. not in [qc[2], qc[3]] and abs(eps) < 0.1 else 0.
s_l[2][5] = '$\pi_{13} \pm i\pi_{14}$'
# pi_25 +/- i*pi_26
m_c[6]    = np.sqrt((qc[2]*qm[1] + qc[5]*qm[2])*F + 2*xi[6]*(e*eps*F)**2) if 0. not in [qc[2], qc[5]] and abs(eps) < 0.1 else 0.
s_l[2][6] = '$\pi_{25} \pm i\pi_{26}$'
# pi_27 +/- i*pi_28
m_c[7]    = np.sqrt((qc[3]*qm[1] + qc[4]*qm[2])*F + 2*xi[7]*(e*eps*F)**2) if 0. not in [qc[3], qc[4]] and abs(eps) < 0.1 else 0.
s_l[2][7] = '$\pi_{27} \pm i\pi_{28}$'
# pi_32 +/- i*pi_33
m_c[8]    = np.sqrt((qc[4]*qm[2] + qc[5]*qm[2])*F + 2*xi[8]*(e*eps*F)**2) if 0. not in [qc[4], qc[5]] and abs(eps) < 0.1 else 0.
s_l[2][8] = '$\pi_{32} \pm i\pi_{33}$'
    
# Mask zero-valued / disabled species in arrays
m_r = np.ma.masked_where(m_r == 0.0, m_r, copy=True)
m_n = np.ma.masked_where(m_n == 0.0, m_n, copy=True)
m_c = np.ma.masked_where(m_c == 0.0, m_c, copy=True)
#np.ma.set_fill_value(m_r, 1.0)
#np.ma.set_fill_value(m_n, 1.0)
#np.ma.set_fill_value(m_c, 1.0)
r_mask = np.ma.getmask(m_r)
n_mask = np.ma.getmask(m_n)
c_mask = np.ma.getmask(m_c)
N_r = m_r.count()
N_n = m_n.count()
N_c = m_c.count()
masks = np.array([r_mask, n_mask, c_mask], dtype=object)

In [6]:
# Initial Conditions
A_0    = 0.1
Adot_0 = 0.0
A_pm   = +1       # specify A± case (+1 or -1)
A_sens = 1.0      # sensitivity for classification of resonance conditions

In [7]:
# Time domain
t_span = [0, 300]   # Units of 1/m_u
#t_N    = 300       # Number of timesteps
t_N = 100
t_sens = 0.1       # sensitivity for calculating time-averaged values

# k values
use_k_eq_0 = False     # Toggle whether or not k = 0 is included in the numerics (div. by 0 error possible if on)
k_min = 0 if use_k_eq_0 else 1
k_max = 100
k_span = [k_min, k_max]  # TODO: replace with the appropriate values
k_res = 1                            # k-mode granularity
k_N = int((1./k_res)*max((k_max - k_min), 0) + 1)    # Number of k-modes

In [8]:
## Parameters of model
manual_set = False       # Toggle override mass definitions
unitful_masses = True    # Toggle whether to provide unitful [eV] masses vs. mass-ratio [m_unit] values for calculations (Default: True)
unitful_k = False        # Toggle whether k values are defined unitfully [eV] vs. units of mass-ratio [m_unit] (Default: False)
#m_unit = m_scale
m_unit = np.min([np.min(m_i) for m_i in (m_r.compressed(), m_n.compressed(), m_c.compressed()) if len(m_i) > 0])

## masses for real, complex, and charged species (given in units of eV)
if manual_set:               # Override (for testing purposes)
    m_scale = 1e-8 if unitful_masses else 1.
    m_r = np.array([1.])*m_scale
    m_n = np.array([3.])*m_scale
    m_c = np.array([5.])*m_scale
    m = np.array([m_r, m_n, m_c], dtype=object, copy=True)
    m_raw = m
else:
    m = np.array([m_r.compressed(), m_n.compressed(), m_c.compressed()], dtype=object, copy=True)
    m_raw = m
    m *= (1. if unitful_masses else 1./m_unit) # Ensure m is provided in desired units
    if unitful_masses and not use_natural_units: 
        m      *= (1./c**2) # WIP
        m_unit *= (1./c**2)
        
# Turn off irrelevant constants
if N_r <= 0 and N_n <= 0:
    L4 = -1                   # Turn off Lambda_4 if there are no surviving neutral (real and complex) species
if N_c <= 0:
    L3 = -1                   # Turn off Lambda_3 if there are no surviving charged species


## local DM densities for each species, assume equal mix for now.
# TODO: More granular / nontrivial distribution of densities? Spacial dependence? Sampling?
norm_subdens = True
if norm_subdens:
    p_loc = p_t/3.
else:
    p_loc = p_t

## Override (for testing purposes)
if manual_set:
    p_r = np.array([1./max(1., N_r)])
    p_n = np.array([1./max(1., N_n), 1./max(1., N_n)])
    p_c = np.array([0.])
    #p_r = np.array([1.])
    #p_n = np.array([1.])
    #p_c = np.array([1.])
    
    p = np.array([p_r*p_loc, p_n*p_loc, p_c*p_loc], dtype=object)
else:
    p_r  = np.ma.masked_where(masks[0], np.full_like(m_r, 1./max(1., N_r), dtype=float), copy=True)
    p_n  = np.ma.masked_where(masks[1], np.full_like(m_n, 1./max(1., N_n), dtype=float), copy=True)
    p_c  = np.ma.masked_where(masks[2], np.full_like(m_c, 1./max(1., N_c), dtype=float), copy=True)
    np.ma.set_fill_value(p_r, 0.0)
    np.ma.set_fill_value(p_n, 0.0)
    np.ma.set_fill_value(p_c, 0.0)

    p = np.array([p_r.compressed()*p_loc, p_n.compressed()*p_loc, p_c.compressed()*p_loc], dtype=object, copy=True)

## initial (local) DM amplitudes for each species, optional units of [eV/c]
amps = np.array([np.array([np.sqrt(2 * p[s][i]) / m[s][i] if np.abs(m[s][i]) > 0. else 0. for i in range(len(m[s]))], dtype=float) for s in range(len(m))], dtype=object, copy=True)
amps_raw = amps
# normalize/rescale amplitudes by dividing by amp of pi_0?
unitful_amps = unitful_masses  # Assume amplitudes given are in units of [eV] if masses are as well (otherwise eV^2/m_unit)
rescale_amps = use_mass_units if unitful_amps else not(use_mass_units)
if rescale_amps and unitful_amps:
    amps /= m_unit          # convert to units of [m_unit] instead of [eV]
if unitful_amps and not(rescale_amps) and not(use_natural_units):
    amps *= np.sqrt((h/c)**3) # (TODO: WIP)

## local phases for each species (to be sampled)
d        = np.array([np.zeros_like(m[0], dtype=float), np.zeros_like(m[1], dtype=float), np.zeros_like(m[2], dtype=float)], dtype=object)   # (0, 2pi)
d_sample = True

## global phase for neutral complex species (to be sampled)
Th        = np.array([np.zeros_like(m[0], dtype=float), np.zeros_like(m[1], dtype=float), np.zeros_like(m[2], dtype=float)], dtype=object)   # (0, 2pi)
Th_sample = True

if debug_level > 2:
    print('m_unit:  ', m_unit)
    if debug_level > 3:
        print('m (raw):\n', m_raw)
        print('m (out):\n', m)
    else:
        print('m:\n', m)
    if debug_level > 3:
        print('amps (raw):\n', amps_raw)
        print('amps (out):\n', amps)
    else:
        print('amps:\n', amps)

#trim_masked_arrays = lambda arr: np.ma.masked_where(len(arr) <= 0, arr)
trim_masked_arrays = lambda arr: np.array([np.array(arr_i, dtype=float) if len(arr_i) > 0 else np.array([], dtype=float) for arr_i in arr], dtype=object)
m    = trim_masked_arrays(m)
p    = trim_masked_arrays(p)
amps = trim_masked_arrays(amps)
Th   = trim_masked_arrays(Th)
d    = trim_masked_arrays(d)

In [9]:
# Sample phases from normal distribution, between 0 and 2pi
cosine_form = True
mu  = np.pi      # mean
sig = np.pi / 3  # standard deviation

# Modulo 2pi to ensure value is within range
shift_val = np.pi if cosine_form else 0.
if d_sample:
        #d  = np.array([np.ma.masked_where(masks[i], np.mod(np.random.normal(mu, sig, len(d_i)), 2*np.pi)).compressed() for i,d_i in enumerate(d)], dtype=object)
        d  = np.array([np.mod(np.random.normal(mu, sig, len(d_i)) + shift_val, 2*np.pi) for i,d_i in enumerate(d)], dtype=object)
if Th_sample:
        #Th = np.array([np.ma.masked_where(masks[i], np.mod(np.random.normal(mu, sig, len(Th_i)), 2*np.pi)).compressed() for i,Th_i in enumerate(Th)], dtype=object)
        Th = np.array([np.mod(np.random.normal(mu, sig, len(Th_i)) + shift_val, 2*np.pi) for i,Th_i in enumerate(Th)], dtype=object)

In [10]:
## Define the system of ODEs
# TODO: Finish defining time-dependent functions P(t), B(t), C(t), D(t) here
#       - Verify they are the correct form, double check all signs and factors of 2
# NOTE: Using cosine definitons for amplitudes
#       i = {0,1,2} correspond to {pi_0, pi, pi_±} respectively

# Toggle whether mass-energy values should be computed in units of eV (False) or pi-axion mass (True)
# (by default, k is defined in units of [m_u] whereas m is defined in units of [eV], so their scaling logic is inverted)
rescale_m      = use_mass_units if unitful_masses else not(use_mass_units)
rescale_k      = not(rescale_m) if unitful_masses else rescale_m
m0 = 1. if not rescale_m else (1./m_unit if unitful_masses else m_unit)         # [m_u] <--> [eV]
k0 = m_unit if rescale_k else 1.

# Rescale all eV unit constants to unit mass
#rescale_consts = False
rescale_consts = rescale_m if unitful_masses else not(rescale_m)
L3_sc = abs(L3) if not rescale_consts or L3 < 0 else L3 / m_unit
L4_sc = abs(L4) if not rescale_consts or L4 < 0 else L4 / m_unit
F_sc  = abs(F)  if not rescale_consts or F < 0  else  F / m_unit

# Shorthand helper function for oscillatory time-dependent terms
# (time is assumed to be given in units of [1/m_u])
phi = lambda t, s, i, m=m, d=d, M=(1./m_unit if unitful_masses else 1.): (m[s][i]*M)*t + d[s][i]
#phi = lambda t, s, i, m=m, d=d, M=(1./m_unit if unitful_masses and not rescale_m else 1.): (m[s][i]*M)*t + d[s][i]

units = {'c': 1 if not unitful_c else 'cm/s', 'h': 1 if not unitful_h else 'eV/Hz', 'G': 1 if not unitful_G else 'cm^5/(eV s^4)', 
         'm': 'm_u' if not((unitful_masses and not rescale_m) or (not unitful_masses and rescale_m)) else 'eV/c^2' if unitful_c else 'eV',
         'k': 'm_u' if not rescale_k else 'eV/c' if unitful_c else 'eV',
         'p': 'eV/cm^3' if p_unit == 1. else 'eV^4' if not(unitful_c or unitful_h) else 'eV^4/(hc)^3',
         'amp': 'm_u' if (rescale_amps and unitful_amps) else 'eV^2/m_u^2' if (rescale_amps and not unitful_amps) else 'eV' if unitful_amps and use_natural_units else 'eV/c',
         'Lambda': 'm_u' if rescale_consts else 'eV/c^2' if unitful_c else 'eV',
         'lambda': 1,
         'F': 1 if F < 0 else 'm_u' if rescale_consts else 'eV/c^2' if unitful_c else 'eV',
         't': '1/m_u',
         'Theta': 'π',
         'delta': 'π'}

pp_param = lambda p, p_u=1., n=2, pfx='  ', notn='e': str(pfx)+('\n'+str(pfx)).join(['m_(%s): %s' % (s_label, ' | '.join([('%.'+('%d' % n)+str(notn)) % (p_s_i*p_u) for p_s_i in p_s]) if len(p_s) > 0 else 'N/A') for p_s, s_label in zip(p, ['0','π','±'])])
if debug_level > 2:
    print('use_mass_units: %5s' % str(use_mass_units), '||', 'use_natural_units:', use_natural_units)
    if debug_level > 3 and not use_natural_units:
        for c_sw, c_name, c_val in zip([unitful_c, unitful_h, unitful_G], ['c', 'h', 'G'], [c, h, G]):
            print('unitful_'+c_name+':', c_sw, '   | ', c_name+' =', str(1 if units[c_name] == 1 else '%.3e [%s]' % (c_val, units[c_val])))
    print('----------------------------------------------------')
if debug_level > 3:
    print('unitful_masses:', '%5s' % str(unitful_masses), '| [m_u]' if not unitful_masses else '| [%s]' % 'eV')
    print('rescale_m:     ', '%5s' % str(rescale_m),      ''        if not rescale_m else '| [%s] -> [%s]' % (('eV','m_u') if unitful_masses else ('m_u','eV')))
    print('unitful_k:     ', '%5s' % str(unitful_k),      '| [m_u]' if not unitful_k else '| [%s]' % 'eV')
    print('rescale_k:     ', '%5s' % str(rescale_k),      ''        if not rescale_k else '| [%s] -> [%s]' % (('eV','m_u') if unitful_k      else ('m_u','eV')))
    print('unitful_amps:  ', '%5s' % str(unitful_amps),   '| [eV]'  if unitful_amps  else '| [eV^2/m_u]')
    print('rescale_amps:  ', '%5s' % str(rescale_amps),   '' if not rescale_amps     else '| [%s] -> [%s]' % (('eV','m_u') if unitful_amps   else ('eV^2/m_u','eV^2/m_u^2')))
    print('rescale_consts:', '%5s' % str(rescale_consts), '' if not rescale_consts   else '| [%s] -> [%s]' % (('eV','m_u') if unitful_masses else ('eV', 'eV/m_u')))
    print('----------------------------------------------------')
if debug_level >= 0:
    print('m_dQCD = %.0e [eV%s]' % (m_scale, '' if use_natural_units else '/c^2'))
    if units['m'] == 'm_u': print('m_u = %.3e [eV%s]' % (m_unit, '' if use_natural_units else '/c^2'))
    print('m [' + units['m'] + ']\n'      + pp_param(m, m0, 3))
    print('rho [' + units['p'] + ']\n'    + pp_param(p, n=3))
    print('amp [' + units['amp'] + ']\n'  + pp_param(amps, n=3))
    print('Theta [π]\n'                   + pp_param(Th, 1. / np.pi, 2, notn='f'))
    print('delta [π]\n'                   + pp_param(d,  1. / np.pi, 2, notn='f'))
    
# Define functions to clean up differential equation representation
P = lambda t, l3=l3, L3=L3_sc, l4=l4, L4=L4_sc, eps=eps, amps=amps, m=m, M=m0, d=d, Th=Th, phi=phi, np=np: \
            2*l3/(L3**2) * eps**2 * (np.sum([amps[2][i]*amps[2][j] * np.cos(phi(t,2,i))*np.cos(phi(t,2,j)) * np.cos(Th[2][i]-Th[2][j]) \
                                             for i in range(len(m[2])) for j in range(len(m[2]))], axis=0)) + \
            2*l4/(L4**2) * eps**2 * (np.sum([amps[1][i]*amps[1][j] * np.cos(phi(t,1,i))*np.cos(phi(t,1,j)) * np.cos(Th[1][i]-Th[1][j]) \
                                             for i in range(len(m[1])) for j in range(len(m[1]))], axis=0) + \
                                     np.sum([amps[0][i]*amps[0][j] * np.cos(phi(t,0,i))*np.cos(phi(t,0,j)) \
                                             for i in range(len(m[0])) for j in range(len(m[0]))], axis=0) + \
                                     2*np.sum([amps[0][i]*amps[1][j] * np.cos(phi(t,0,i))*np.cos(phi(t,1,j)) * np.cos(Th[1][j]) \
                                             for i in range(len(m[0])) for j in range(len(m[1]))], axis=0))

B = lambda t, l3=l3, L3=L3_sc, l4=l4, L4=L4_sc, eps=eps, amps=amps, m=m, M=m0, d=d, Th=Th, phi=phi, np=np: \
            (-1)*2*l3/(L3**2) * eps**2 * (np.sum([amps[2][i]*amps[2][j] * np.cos(Th[2][i]-Th[2][j])  * \
                                                  ((m[2][i]*M) * np.sin(phi(t,2,i)) * np.cos(phi(t,2,j)) + \
                                                   (m[2][j]*M) * np.cos(phi(t,2,i)) * np.sin(phi(t,2,j))) \
                                                  for i in range(len(m[2])) for j in range(len(m[2]))], axis=0)) + \
            (-1)*2*l4/(L4**2) * eps**2 * (np.sum([amps[0][i]*amps[0][j] * ((m[0][i]*M) * np.sin(phi(t,0,i)) * np.cos(phi(t,0,j)) + \
                                                                           (m[0][j]*M) * np.cos(phi(t,0,i)) * np.sin(phi(t,0,j))) \
                                                  for i in range(len(m[0])) for j in range(len(m[0]))], axis=0) + \
                                          np.sum([amps[1][i]*amps[1][j] * np.cos(Th[1][i]-Th[1][j])  * \
                                                  ((m[1][i]*M) * np.sin(phi(t,1,i)) * np.cos(phi(t,1,j)) + \
                                                   (m[1][j]*M) * np.cos(phi(t,1,i)) * np.sin(phi(t,1,j))) \
                                                  for i in range(len(m[1])) for j in range(len(m[1]))], axis=0) + \
                                          np.sum([np.abs(amps[0][i]*amps[1][j]) * np.cos(Th[1][j]) * \
                                                  ((m[0][i]*M) * np.sin(phi(t,0,i)) * np.cos(phi(t,1,j)) + \
                                                   (m[1][j]*M) * np.cos(phi(t,0,i)) * np.sin(phi(t,1,j))) \
                                                  for i in range(len(m[0])) for j in range(len(m[1]))], axis=0))

C = lambda t, pm, l1=l1, F=F_sc, eps=eps, amps=amps, m=m, M=m0, d=d, phi=phi, np=np: \
            (-1) * pm * (2*l1 / F) * eps**2 * np.sum([amps[0][i] * (m[0][i]*M) * np.sin(phi(t,0,i)) \
                                                      for i in range(len(m[0]))], axis=0)

D = lambda t, l2=l2, e=e, eps=eps, amps=amps, m=m, M=m0, d=d, Th=Th, phi=phi, np=np: \
            l2 * eps**2 * e**2 * np.sum([amps[2][i]*amps[2][j] * np.cos(phi(t,2,i))*np.cos(phi(t,2,j)) * np.cos(Th[2][i]-Th[2][j]) \
                                         for i in range(len(m[2])) for j in range(len(m[2]))], axis=0)

override_coefficients = False
if override_coefficients:
    #P = lambda t: np.float64(0)
    #B = lambda t: np.float64(0)
    #D = lambda t: np.float64(0)
    #C = lambda t, pm: np.float64(0)
    override_coefficients = True
    
def local_system(t, y, k, params, P=P, B=B, C=C, D=D, A_pm=A_pm, bg=1, k0=k0):
    # System of differential equations to be solved (bg = photon background)
    dy0dt = y[1]
    dy1dt = -1./(bg + P(t)) * (B(t)*y[1] + (C(t, A_pm)*(k*k0) + D(t))*y[0]) - (k*k0)**2*y[0]
    return [dy0dt, dy1dt]

m_dQCD = 1e-40 [eV]
m_u = 1.414e-07 [eV]
m [m_u]
  m_(0): 1.000e+00 | 1.414e+00
  m_(π): 1.225e+00
  m_(±): N/A
rho [eV^4]
  m_(0): 3.177e+07 | 3.177e+07
  m_(π): 6.353e+07
  m_(±): N/A
amp [m_u]
  m_(0): 3.985e+17 | 2.818e+17
  m_(π): 4.602e+17
  m_(±): N/A
Theta [π]
  m_(0): 1.84 | 0.31
  m_(π): 0.10
  m_(±): N/A
delta [π]
  m_(0): 0.23 | 1.94
  m_(π): 0.33
  m_(±): N/A


In [ ]:
# Solve the differential equation for each k

k_values, k_step = np.linspace(k_span[0], k_span[1], k_N, retstep=True)
#k_values = np.linspace(1./100, 10, 100)

# Initialize an array to store the solutions
t, t_step = np.linspace(t_span[0], t_span[1], t_N, retstep=True)  # Array of times at which to evaluate, t > 0
#t = t[1:]

# Classification sensitivity threshold
res_con = 1000
#res_con = max(100,1./A_sens)

# Set all parameters on the backend
parameters = {'e': e, 'F': F, 'p_t': p_t, 'eps': eps, 'L3': L3, 'L4': L4, 'l1': l1, 'l2': l2, 'l3': l3, 'l4': l4, 'res_con': res_con,
              'A_0': A_0, 'Adot_0': Adot_0, 'A_pm': A_pm, 't_sens': t_sens, 'A_sens': A_sens,
              'qm': qm, 'qc': qc, 'dqm': dqm, 'eps_c': eps_c, 'xi': xi, 'm_0': m_unit, 'm_scale': m_scale, 'p_unit': p_unit,
              'm': m, 'm_r': m_r, 'm_n': m_n, 'm_c': m_c, 'p': p, 'p_r': p_r, 'p_n': p_n, 'p_c': p_c, 'amps': amps, 'd': d, 'Th': Th,
              'unitful_m': unitful_masses, 'rescale_m': rescale_m, 'unitful_amps': unitful_amps, 'rescale_amps': rescale_amps, 
              'unitful_k': unitful_k, 'rescale_k': rescale_k, 'rescale_consts': rescale_consts, 'h': h, 'c': c, 'G': G}
params = set_params(parameters, t_min=t_span[0], t_max=t_span[1], t_N=t_N, k_min=k_span[0], k_max=k_span[1], k_N=k_N)

# Solve the system, in parallel
os.environ['NUMEXPR_MAX_THREADS'] = '32'
solutions, params, time_elapsed, timestr = solve_system(local_system, jupyter=True, parallelize=True, num_cores=32)
print(timestr)

In [ ]:
## Identify the k mode with the greatest peak amplitude, and the mode with the greatest average amplitude
# k_func : apply [func] on the time-series for each k mode, e.g. max or mean
k_func = lambda func: np.array([k_fval for k_fi, k_fval in enumerate([func(np.abs(solutions[k_vi][0][:])) for k_vi, k_v in enumerate(k_values)])])
# k_sens : apply [k_func] but limit the time-series by [sens], e.g. sens = 0.1 to look at the first 10%. Negative values look at the end of the array instead.
w_min  = lambda sens: int(t_N*(1./2)*np.abs(((1. - sens)*np.sign(sens) + (1. - sens))))  # min value / left endpoint (of the sensitivity window over which to average)
w_max  = lambda sens: int(t_N*(1./2)*np.abs(((1. + sens)*np.sign(sens) + (1. - sens))))  # max value / right endpoint
k_sens = lambda func, sens: np.array([k_fval for k_fi, k_fval in enumerate([func(np.abs(solutions[k_vi][0][w_min(sens):w_max(sens)])) for k_vi, k_v in enumerate(k_values)])])
#k_sens = lambda func, sens: np.array([k_fval for k_fi, k_fval in enumerate([func(np.abs(solutions[k_vi][0][int(t_span[1]*(1. - sens)):])) for k_vi, k_v in enumerate(k_values)])])
# k_ratio: apply [k_func] to each k mode and then return the ratio of the final vs. initial ampltidues (sensitive to a windowed average specified by [sens])
k_ratio = lambda func, t_sens, A_sens: np.array([k_f/k_i for k_f, k_i in zip(k_sens(func, t_sens), k_sens(func, -t_sens))])
# k_class: softly classify the level of resonance according to the final/initial mode amplitude ratio, governed by [func, t_sens, and A_sens]
k_class = lambda func, t_sens, A_sens: np.array(['damp' if k_r <= 0.9 else 'none' if k_r <= (1. + np.abs(A_sens)) else 'semi' if k_r <= res_con else 'res' for k_r in k_ratio(func, t_sens, A_sens)])

# k mode(s) with the largest contributions to overall number density growth
k_peak = k_values[np.ma.argmax(k_func(max))]
k_mean = k_values[np.ma.argmax(k_func(np.ma.mean))]

if debug_level > 0:
    print('max (peak) k mode: '+str(k_peak))
    print('max (mean) k mode: '+str(k_mean))

# Plot the solution
signstr = {1: '+', -1: '-', 0: '±'}
#k_samples = np.geomspace(1,len(k_values),num=5)
k_samples = [i for i, k_i in enumerate(k_values) if k_i in [0,1,10,50,100,150,200,500,k_peak,k_mean]]
times = t
plot_Adot = True

xdim = 5
if plot_Adot:
    ydim = 3 
else:
    ydim = 2

plt.figure(figsize=(4*xdim, 4*ydim))

#plt.subplot(2,1,1)
plt.subplot2grid((ydim,xdim), (0,0), colspan=3)
for k_sample in k_samples:
    k_s = int(k_sample)
    #print(solutions[k_s, 0])
    plt.plot(times, solutions[k_s][0], label='k='+str(k_values[k_s]))
plt.title('Evolution of the mode function $A_{'+signstr[0]+'}$(k)')
plt.xlabel('Time [%s]' % units['t'])
plt.ylabel('$A_{'+signstr[0]+'}$(k)')
plt.yscale('log')
plt.legend()
plt.grid()

#plt.subplot(2,1,2)
plt.subplot2grid((ydim,xdim), (1,0), colspan=3)
plt.plot(times, [sum([np.abs(solutions[i][0][t_i])**2 for i in range(len(k_values))]) for t_i in range(len(times))])
plt.title('Evolution of the (total) power for $A_{'+signstr[0]+'}$')
plt.xlabel('Time [%s]' % units['t'])
plt.ylabel('$|A_{'+signstr[0]+'}|^2$')
plt.yscale('log')
plt.grid()


if plot_Adot:
    #plt.subplot(2,1,2)
    plt.subplot2grid((ydim,xdim), (2,0), colspan=3)
    plt.plot(times, [sum([solutions[i][1][t_i] for i in range(len(k_values))]) for t_i in range(len(times))], color='g', label='total')
    plt.plot(times, [solutions[list(k_values).index(k_mean)][1][t_i] for t_i in range(len(times))], color='y', label='k = %d (mean)' % k_mean)
    plt.plot(times, [solutions[list(k_values).index(k_peak)][1][t_i] for t_i in range(len(times))], color='orange', label='k = %d (peak)' % k_peak)
    plt.title('Evolution of the (total) change in amplitude for A'+signstr[0])
    plt.xlabel('Time [%s]' % units['t'])
    plt.ylabel('$\dot{A}_{'+signstr[0]+'}$')
    #plt.yscale('log')
    plt.legend()
    plt.grid()

#print(len(d[0]))
    
textstr1, textstr2 = get_text_params(units_in=units)

plt.subplot2grid((ydim,xdim), (max(0,ydim-3),3), rowspan=2)
plt.text(0.15, 0.2, textstr1, fontsize=14)
plt.axis('off')

plt.subplot2grid((ydim,xdim), (max(0,ydim-3),4), rowspan=2)
plt.text(0, 0.2, textstr2, fontsize=14)
plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Plot the occupation numbers (TODO: Verify units in below equation)

w_unit = np.float64(4.555e25) # 2πc/hbar [(Hz/eV)*(cm/s)]
w = lambda i, k0=k0, w0=w_unit: np.abs(k_values[i]*(k0*w0))
n = lambda i: (w(i)/2) * (((np.square(np.abs(solutions[i][1])))/(np.square(w(i)))) + np.square(np.abs(solutions[i][0]))) - (1/2)
times = t
scale_n = True

if debug_level > 1:
    print('k0:  ',     k0)
    print('k_1:  ',    k_values[0])
    print('w_unit:  ', w_unit)

#k_samples = np.geomspace(1,len(k_values),num=5)
k_samples = [i for i, k_i in enumerate(k_values) if k_i in [0,1,10,20,50,75,100,125,150,175,200,500,k_peak,k_mean]]
plt.figure(figsize=(20, 8))

plt.subplot2grid((2,5), (0,0), colspan=3)
for k_sample in k_samples:
    k_s = int(k_sample)
    plt.plot(times, n(k_s), label='k='+str(k_values[k_s]))
plt.title('Occupation numbers')
plt.xlabel('Time [%s]' % units['t'])
#plt.xlim(0,0.2)
plt.ylabel('n_k/$n_0$' if scale_n else 'n_k')
plt.yscale('log'); #plt.ylim(bottom = 0.1)
plt.legend()
plt.grid()

n_tot = np.array([sum([n(i)[t_i] for i in range(len(k_values))]) for t_i in range(len(times))])
if scale_n:
    n_tot /= abs(n_tot[0])
    n_tot += max(0, np.sign(n_tot[0]))  # Placeholder fix for negative n

#tot_res = 'resonance' if n_tot[-1]/n_tot[0] > 100 else 'none'
tot_res = 'resonance' if sum(k_ratio(np.ma.mean, t_sens, A_sens)) > res_con else 'none'
t_res_i = np.ma.argmax(np.array(n_tot) > res_con)
t_res   = t[t_res_i]
n_res   = n_tot[t_res_i]
parameters['res_class'] = tot_res
parameters['t_res'] = t_res
#n_res = res_con*sum(k_sens(np.mean, -t_sens))

#with plt.xkcd():
plt.subplot2grid((2,5), (1,0), colspan=3)
#fig,ax = plt.subplots()
#plt.plot(np.ma.masked_where(t >= t_res, times), np.ma.masked_where(np.array(n_tot) > res_con*sum(k_sens(np.mean, -t_sens)), n_tot), label='none', color='grey')
plt.plot(np.ma.masked_greater_equal(times, t_res), n_tot, label='none', color='grey')
plt.plot(np.ma.masked_less(times, t_res), n_tot, label='resonance', color='blue')
plt.title('Occupation Number (total)')
plt.xlabel('Time [%s]' % units['t'])
#plt.xlim(0,0.1)
plt.ylabel('n')
plt.yscale('log')
plt.legend()
plt.grid()

textstr1, textstr2 = get_text_params(units_in=units)

plt.subplot2grid((2,5), (0,3), rowspan=2)
plt.text(0.15, 0.2, textstr1, fontsize=14)
plt.axis('off')

plt.subplot2grid((2,5), (0,4), rowspan=2)
plt.text(0, 0.2, textstr2, fontsize=14)
plt.axis('off')

plt.tight_layout()
plt.show()

print('n_tot in range [%.2e, %.2e]' % (min(n_tot),max(n_tot)))
if 'res' in tot_res:
    print('resonance classification begins at t = %.2f, n = %.2e' % (t_res, n_res))

In [ ]:
# Compare model coefficient values over time
plt.figure(figsize = (20,7))
times = t
c_ranges = []
signstr = {1: '+', -1: '-', 0: '±'}

plt.subplot2grid((2,5), (0,0), colspan=3, rowspan=2)

for c_func, l_root, sign in zip([P, B, C, C, D], 
                                ['P(t)', 'B(t)', 'C_{%s}(t)' % signstr[+1], 'C_{%s}(t)' % signstr[-1], 'D(t)'], 
                                [0, 0, +1, -1, 0]):
    if l_root[0] == 'C':
        c_t = c_func(times, sign)
    else:
        c_t = c_func(times)
    
    if type(c_t) is np.float64:
        label   = '[[$' + l_root + '$]]'
        c_t     = np.ma.array(c_t + np.full(len(t), 0.0, dtype=float), mask=True)
        c_range = (np.nan, np.nan)
    else:
        label   = '$' + l_root + '$'
        c_range = (min(c_t), max(c_t))
        
    plt.plot(times, c_t, label=label)
    c_ranges.append(l_root[0] + '(t) range: [%.1e, %.1e]' % c_range + (' for + case ' if sign > 0 else ' for - case ' if sign < 0 else ''))

plt.xlabel('Time [%s]' % units['t'])
plt.yscale('log')
plt.grid()
plt.legend()

textstr1, textstr2 = get_text_params(units_in=units)

plt.subplot2grid((2,5), (0,3), rowspan=2)
plt.text(0.15, 0.1, textstr1, fontsize=14)
plt.axis('off')

plt.subplot2grid((2,5), (0,4), rowspan=2)
plt.text(0, 0.1, textstr2, fontsize=14)
plt.axis('off')

plt.tight_layout()
plt.show()

for c_range_str in c_ranges:
    print(c_range_str)

In [ ]:
# Compare Alpha(t) and Beta(t) for a sampling of k values
#   for EoM of the form: A_k'' + [Beta]A_k' + [Alpha(k)]A_k = 0
times = t

Alpha = lambda t, k, k0=k0: ((C(t, A_pm)*(k*k0) + D(t)) / (1. + P(t))) + (k*k0)**2
Beta  = lambda t: B(t) / (1. + P(t))

plt.figure(figsize = (20,7))
plt.subplot2grid((2,5), (0,0), colspan=3, rowspan=2)

#k_samples = np.geomspace(1,len(k_values),num=5)
k_samples = [i for i, k_i in enumerate(k_values) if k_i in [0,1,10,25,50,75,100,150,200,500]]

if all([type(Alpha(times, k_s)) is np.float64 for k_s in k_samples]):
    label_a = r'[[$\alpha(t)$]]'
    alpha_s = np.ma.array(alpha_s + np.full(len(t), 0.0, dtype=np.float64), mask=True)
    plt.plot(times, alpha_s, label=label_a)
else:
    for k_sample in k_samples:
        k_s = int(k_sample)
        alpha_s = Alpha(times, k_values[k_s])
        if type(alpha_s) is np.float64:
            label_a = r'[[$\alpha_k(t)$  k=%d]]' % k_values[k_s]
            alpha_s = np.ma.array(alpha_s + np.full(len(t), 0.0, dtype=np.float64), mask=True)
            plt.plot(times, alpha_s, label=label_a)
        else:
            label_a = r'$\alpha_k(t)$  k=%d' % k_values[k_s]
            plt.plot(times, alpha_s, label=label_a)

beta_s = Beta(times)
if type(beta_s) is np.float64:
    label_b = r'[[$\beta(t)$]]'
    beta_s  = np.ma.array(beta_s + np.full(len(t), 0.0, dtype=np.float64), mask=True)
else:
    label_b = r'$\beta(t)$'
plt.plot(times, beta_s, label=label_b)

plt.grid()
plt.xlabel('Time [%s]' % units['t'])
plt.yscale('log')
plt.legend()

textstr1, textstr2 = get_text_params(units_in=units)

plt.subplot2grid((2,5), (0,3), rowspan=2)
plt.text(0.15, 0.1, textstr1, fontsize=14)
plt.axis('off')

plt.subplot2grid((2,5), (0,4), rowspan=2)
plt.text(0, 0.1, textstr2, fontsize=14)
plt.axis('off')

plt.tight_layout()
plt.show()

if all([type(Alpha(times, k_s)) is np.float64 for k_s in k_samples]):
    print('Alpha(t) range: [%.1e, %.1e] for all k' % (np.nan, np.nan))
else:
    for k_s in k_samples:
        if type(Alpha(times, k_values[k_s])) is not np.float64:
            print('Alpha(t) range: [%.1e, %.1e] when k = %d' % (min(Alpha(times, k_values[k_s])), max(Alpha(times, k_values[k_s])), k_values[k_s]))
if type(Beta(times)) is np.float64:
    print('Beta(t) range: [%.1e, %.1e]' % (np.nan, np.nan))
else:
    print('Beta(t) range: [%.1e, %.1e]' % (min(Beta(times)), max(Beta(times))))

In [ ]:
if debug_level > 3:
    print('params:\n', params, '\n')

# store max, all-time mean, and late-time mean for each k-mode
params['k_peak_arr']  = k_func(max)
params['k_mean_arr']  = k_func(np.mean)
params['k_sens_arr']  = k_sens(np.mean, t_sens)
params['k_class_arr'] = k_class(np.mean, t_sens, A_sens)

params['class'] = tot_res

if debug_level > 2:
    print('params[\'class\']:\n', params['class'])
    if debug_level > 6:
        print('params[\'k_class_arr\']:\n', params['k_class_arr'])
        print('k_ratio:\n', k_ratio(np.mean, t_sens, A_sens))

class_colors = {'none': 'lightgrey', 'damp': 'darkgrey', 'semi': 'blue', 'res': 'red'}


# Resonant k-mode (units of quark mass -> eV) to frequency (Hz) conversion functions
#m_0 = qm[0]
#m_0 = m_unit if (unitful_masses and not rescale_m) or (not unitful_masses and rescale_m) else 1.
#k_to_Hz = lambda kr, m_0=m_0: 1.2e4 * (kr * m_0 / 10e-10)
#Hz_to_k = lambda fr, m_0=m_0: (10e-10 / m_0) * (fr / 1.2e4)

# E^2 = p^2c^2 + m^2c^4
# Assuming k, m are given in units of eV/c and eV/c^2 respectively
#k_to_Hz = lambda ki, mi=0, m_0=m0, e=e: 1/h * np.sqrt((ki*k0*e)**2 + ((mi*m_0 * e))**2)
k_to_Hz = lambda ki, h=h, e=e: ki * ((k0*e) / h)
#Hz_to_k = lambda fi, mi=0, m_0=m0, e=e: 1/(e*k0) * np.sqrt((h * fi)**2 - ((mi*m_0 * e))**2)
Hz_to_k = lambda fi, h=h, e=e: fi * (h / (k0*e))

# Known observable frequency ranges (Hz)
FRB_range = [100e6, 5000e6]
GRB_range = [3e19, 3e21]

Hz_label = lambda f, pd=pd: pd.cut([f],
                                   [0, 300e6, 3e12, 480e12, 750e12, 30e15, 30e18, np.inf],
                                   labels=['Radio', 'Microwave', 'Infrared', 'Visible', 'UV', 'X-ray', 'Gamma ray'])

plt.figure(figsize = (20,6))
plt.suptitle('Resonance Classification')

ax = plt.subplot2grid((2,4), (0,0), colspan=2, rowspan=2)
plt.scatter(k_values, k_ratio(np.mean, t_sens, A_sens), c=[class_colors[k_c] if k_c in class_colors else 'orange' for k_c in k_class(np.mean, t_sens, A_sens)])
plt.xlabel('$k$')
axT = ax.secondary_xaxis('top', functions=(k_to_Hz, Hz_to_k))
axT.set_xlabel('$f_{\gamma}$ (Hz)')
plt.ylabel('Growth in $n_k$')
plt.yscale('log')
plt.grid()

plt.subplot2grid((2,4), (0,2), colspan=2, rowspan=2)
class_counts = [(np.array(k_class(np.mean, t_sens, A_sens)) == class_label).sum() for class_label in class_colors.keys()]
plt.bar(class_colors.keys(),class_counts,color=class_colors.values())
plt.xlabel('Classification')
plt.ylabel('Count')
plt.grid()

plt.tight_layout()
plt.show()

if 'res' in tot_res and debug_level >= 0:
    Hz_peak = k_to_Hz(k_peak)
    print('peak resonance at k = %d corresponds to photon frequency at %.2e Hz (%s)' % (k_peak, Hz_peak, Hz_label(Hz_peak)[0]))
    if Hz_peak >= FRB_range[0] and Hz_peak <= FRB_range[1]:
        print('possible FRB signal')
    if Hz_peak >= GRB_range[0] and Hz_peak <= GRB_range[1]:
        print('possible GRB signal')

In [ ]:
plt.figure(figsize = (16,12))
#plt.suptitle('ALP Survey Results')

plt.subplot2grid((1,1), (0,0), colspan=1, rowspan=1)
xmin, xmax = (1e-12, 1e7)   # Scale limits
ymin, ymax = (1e-21, 2e-6)   # Scale limits
res_color  = '#b042f5'
plot_masses = True
show_mass_ranges = False

## Interactive mode (WIP)
with plt.ion():
    # Log-scaled axes
    ax = plt.gca()
    ax.minorticks_on()
    ax.set_xlabel('$m_a\quad[eV]$',fontsize=30)
    ax.set_xlim(xmin, xmax)
    ax.set_xscale('log')
    ax.set_ylabel('$|g_{a\gamma}|\quad[GeV^{-1}]$',fontsize=30)
    ax.set_ylim(ymin, ymax)
    ax.set_yscale('log')
    ax.tick_params(axis='both', which='major', labelsize=15)
    ax.tick_params(axis='both', which='minor', labelsize=8)
    ax.set_zorder(3)
    ax.set_facecolor('none')
    #ax.grid('on')
    
    ## Primary data
    ax.set_zorder(3)
    ax.scatter(m_unit, GeV/F, s=1000, c=res_color, marker='*')
    
    ## Secondary Data
    ax2 = plt.gcf().add_subplot()
    ax2.set_xscale('log')
    ax2.set_xlim(xmin, xmax)
    ax2.set_yscale('log')
    ax2.set_ylim(ymin, ymax)
    ax2.axis('off')
    ax2.set_zorder(2)
    # Millicharge line (WIP)
    #eps_logval = 2*np.log10(10*eps)-19.9
    eps_logval = lambda ep: (2.615)*np.log10(10*ep + 0.868)-(20.61)
    eps_scaleval = 2
    if debug_level > 3:
        print('10^eps: ', 10**(eps_logval(eps)))
        print('10^eps * scale: ', 10**(eps_logval(eps)) * eps_scaleval)
    eps_str = ('$\\varepsilon=%s$' % ('%d' if eps == 1 else '%.1f' if eps > 0.01 else '%.1e')) % eps
    ax2.plot(np.geomspace(xmin, xmax), np.geomspace((10**(eps_logval(eps)) * eps_scaleval), (10**(eps_logval(eps)+2.15) * eps_scaleval)), c='magenta', linestyle='solid', label=eps_str)
    # F_pi and Lambda lines (WIP)
    ax2.hlines(GeV / F, xmin, m_unit, colors=res_color, linestyles='dashed', label='$F_{\pi}$')
    for Lval, Lidx in zip([L3, L4], ['3','4']):
        if Lval > 0:
            ax2.hlines(GeV / Lval, xmin, xmax, colors='black', linestyles='dashdot', label='$\Lambda_%s$' % Lidx)
    # m_i lines/blocks for each species (WIP)
    if plot_masses:
        for s_idx, m_s in enumerate(m):
            s_idx_str = '0' if s_idx == 0 else '\pi' if s_idx == 1 else '\pm' if s_idx == 2 else '\text{ERR}'
            for m_idx, m_i in enumerate(m_s):
                m_plot_data = {'x': m_i,
                               'label': '$m_{(%s),%s}$' % (s_idx_str, str(m_idx)),
                               'color': res_color if m_i == m_unit else 'black',
                               'style': 'dashed'}
                if len(m_s) == 1 or m_i == m_unit or not(show_mass_ranges):  # Plot a vertical line
                    ax2.vlines(m_plot_data['x'], ymin, ymax, colors=m_plot_data['color'], linestyles=m_plot_data['style'], label=m_plot_data['label'])
                if len(m_s) > 1 and m_idx <= 0 and show_mass_ranges:         # Plot a horizontal line
                    ax2.axvspan(min(m_s), max(m_s), ymin=0, ymax=1, color='black', label='$m_{(%s)}$' % (s_idx_str), alpha=0.5, visible=show_mass_ranges)
    ax2.legend()
                    
    
    ## Background Image
    survey_img = image.imread('./survey_img_crop.PNG')
    im_ax = ax.twinx()
    im_ax.axis('off')
    im_ax = im_ax.twiny()
    im_ax.axis('off')
    im_ax.imshow(survey_img, extent=[xmin, xmax, ymin, ymax], aspect='auto')

plt.show()

In [ ]:
fit_epsilon_reln = False
if fit_epsilon_reln:
    pts = [(0.1,-19.9),(0.5,-18.6),(1,-17.9)]
    pts_x = [x for x,y in pts]
    pts_y = [y for x,y in pts]

    logfit = lambda t, a, b, c: a*np.log10(10*t+b)+c
    xspace = np.linspace(0.01, 1.5, 150)

    popt, pcov = curve_fit(logfit, np.array(pts_x),np.array(pts_y),p0=(2,0,-19.9))

    plt.plot(pts_x, pts_y, marker='x', linewidth=0, ms=10, color='red')
    plt.plot(xspace, logfit(xspace, a=2, b=0, c=-19.9), color='blue')
    plt.plot(xspace, logfit(xspace, a=popt[0], b=popt[1], c=popt[2]), color='green')

    plt.ylim(-21, -17)
    plt.xlim(0, 1.1)
    plt.yscale('exp')
    plt.grid()
    plt.show()

    print('y = %.3f log_10(10x + %.3f) + %.3f' % (popt[0],popt[1],popt[2]))